In [52]:
import fiona
import rtree
from pyproj import transformer
import geopandas
import pandas
from shapely.geometry import LineString

from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource

In [53]:
# Read in a single strava file. Add functionality for multiple rides later
strava = geopandas.read_file('/home/jenny/Downloads/Afternoon_Ride2.gpx', layer='tracks')
strava = strava.to_crs("EPSG:3857")

In [54]:
# Turn the strava MultiLineString into short LineStrings
for x in strava['geometry'].iloc[0]:
    coords = [y for y in x.coords]
    lines = []
    for i in range(len(coords)-1):
        lines.append(LineString([coords[i], coords[i+1]]))

d = {'col1': range(len(lines)), 'geometry': lines}
strava_gdf = geopandas.GeoDataFrame(d, crs="EPSG:3857")


2729
2730


,col1,geometry
0,0,"LINESTRING (-7916643.327 5218814.881, -7916639..."
1,1,"LINESTRING (-7916639.988 5218816.840, -7916636..."
2,2,"LINESTRING (-7916636.537 5218818.800, -7916633..."
3,3,"LINESTRING (-7916633.197 5218820.910, -7916629..."
4,4,"LINESTRING (-7916629.747 5218822.869, -7916626..."


In [57]:
# Read in the map for the city
import osmnx as ox
#ox.geocode_to_gdf

G = ox.graph_from_place('Cambridge, MA USA', network_type='drive', simplify=True)

# Turn into a gdf in epgs 3857
nodes, ways = ox.utils_graph.graph_to_gdfs(G)
ways = ways.to_crs("EPSG:3857") 


In [63]:
# Add base map for city
output_file("tile.html")

tile_provider = get_provider(CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(x_range=[ways.total_bounds[0], ways.total_bounds[2]], y_range=[ways.total_bounds[1], ways.total_bounds[3]])
p.add_tile(tile_provider)


TileRenderer(id='68280', ...)

In [64]:
def add_gdf_to_map(gdf, p, color, width=2):
    x_coords = []
    y_coords = []

    # display Cambridge roads
    for index, road in gdf.iterrows():
    
        coords = [x for x in road['geometry'].coords]
        x_coords.append([x[0] for x in coords])
        y_coords.append([x[1] for x in coords])


    # make a multilinestring from a set of linestrings
    source = ColumnDataSource( dict(x=x_coords,y=y_coords) )    
    l = p.multi_line( xs='x', ys='y', source=source, color=color, line_width=width)                    


    return p


# Add ways to map
p = add_gdf_to_map(ways, p, 'red')
# Add strava routes to map
p = add_gdf_to_map(strava_gdf, p, 'green', width=5)
show(p)